In [1]:
import pandas as pd
import pyreadr
import numpy as np

In [3]:
#load features data
features = pd.read_csv('../mimic_data/master_dataset.csv')
features.shape

(425087, 120)

In [6]:
icd10 = pd.read_csv('../mimic_data/ALL/icd_list_dataset_v10.csv')

In [46]:
admission = pd.read_csv('../mimic4ed-benchmark-main/data/core/admissions.csv')
ed = pd.read_csv('../mimic4ed-benchmark-main/data/ed/edstays.csv')
diagnosis = pd.read_csv('../mimic4ed-benchmark-main/data/hosp/diagnoses_icd.csv')
diagnosis_code = pd.read_csv('../mimic4ed-benchmark-main/data/hosp/d_icd_diagnoses.csv')
patients = pd.read_csv('../mimic4ed-benchmark-main/data/core/patients.csv')

In [11]:
admission.columns.values

array(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admit_provider_id', 'admission_location',
       'discharge_location', 'insurance', 'language', 'marital_status',
       'race', 'edregtime', 'edouttime', 'hospital_expire_flag'],
      dtype=object)

In [15]:
diagnosis.columns.values

array(['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version'],
      dtype=object)

In [16]:
diagnosis.icd_version.unique()

array([ 9, 10])

In [22]:
included_icd9 = diagnosis[(diagnosis.icd_version == 9) & (diagnosis.icd_code.str.startswith('493'))].icd_code.unique()

In [23]:
included_icd10 = diagnosis[(diagnosis.icd_version == 10) & (diagnosis.icd_code.str.startswith('J45'))].icd_code.unique()

In [25]:
diagnosis_code[(diagnosis_code.icd_version==9) & (diagnosis_code.icd_code.isin(included_icd9))]

,icd_code,icd_version,long_title
5238,49300,9,"Extrinsic asthma, unspecified"
5239,49301,9,Extrinsic asthma with status asthmaticus
5240,49302,9,Extrinsic asthma with (acute) exacerbation
5241,49310,9,"Intrinsic asthma, unspecified"
5243,49312,9,Intrinsic asthma with (acute) exacerbation
5244,49320,9,"Chronic obstructive asthma, unspecified"
5245,49321,9,Chronic obstructive asthma with status asthmat...
5246,49322,9,Chronic obstructive asthma with (acute) exacer...
5247,49381,9,Exercise induced bronchospasm
5248,49382,9,Cough variant asthma


In [27]:
diagnosis_code[(diagnosis_code.icd_version==10) & (diagnosis_code.icd_code.isin(included_icd10))]

,icd_code,icd_version,long_title
26415,J4520,10,"Mild intermittent asthma, uncomplicated"
26416,J4521,10,Mild intermittent asthma with (acute) exacerba...
26417,J4522,10,Mild intermittent asthma with status asthmaticus
26419,J4530,10,"Mild persistent asthma, uncomplicated"
26420,J4531,10,Mild persistent asthma with (acute) exacerbation
26423,J4540,10,"Moderate persistent asthma, uncomplicated"
26424,J4541,10,Moderate persistent asthma with (acute) exacer...
26427,J4550,10,"Severe persistent asthma, uncomplicated"
26428,J4551,10,Severe persistent asthma with (acute) exacerba...
26429,J4552,10,Severe persistent asthma with status asthmaticus


In [36]:
included_data = diagnosis[((diagnosis.icd_version == 9) & (diagnosis.icd_code.str.startswith('493')))|((diagnosis.icd_version == 10) & (diagnosis.icd_code.str.startswith('J45')))]
# included_data = included_data[(included_data.icd_version == 10) & (included_data.icd_code.str.startswith('J45'))]

In [41]:
included_data.hadm_id.values

array([25563031, 21268656, 21577720, ..., 26008899, 25345411, 26785317])

In [42]:
ed[ed.hadm_id.isin(included_data.hadm_id.values)]

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition
73,10001884,24325811.0,33281437,2126-11-03 19:15:00,2126-11-04 12:49:00,F,BLACK/AFRICAN AMERICAN,WALK IN,HOME
74,10001884,24746267.0,35329716,2130-12-27 15:48:00,2130-12-27 22:30:00,F,BLACK/AFRICAN AMERICAN,WALK IN,ADMITTED
75,10001884,24962904.0,31742950,2130-12-06 16:46:00,2130-12-06 22:05:00,F,BLACK/AFRICAN AMERICAN,WALK IN,ADMITTED
76,10001884,25758848.0,38343289,2128-07-15 15:15:00,2128-07-15 20:17:00,F,BLACK/AFRICAN AMERICAN,WALK IN,HOME
77,10001884,26170293.0,38261474,2130-04-15 16:38:00,2130-04-15 22:19:00,F,BLACK/AFRICAN AMERICAN,WALK IN,ADMITTED
...,...,...,...,...,...,...,...,...,...
424765,19992167,25148291.0,36727273,2200-06-02 21:19:00,2200-06-03 02:35:00,F,WHITE,WALK IN,ADMITTED
424766,19992202,20329411.0,36334096,2153-02-25 18:11:00,2153-02-26 01:58:00,F,WHITE,AMBULANCE,ADMITTED
424962,19996832,29420557.0,32130207,2179-02-21 13:29:00,2179-02-22 11:51:00,F,ASIAN,AMBULANCE,HOME
425023,19998350,21130518.0,32922923,2131-04-25 20:34:00,2131-04-26 20:17:00,M,BLACK/AFRICAN AMERICAN,WALK IN,ADMITTED


In [45]:
admission[admission.hadm_id.isin(included_data.hadm_id.values)]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
58,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaN,EW EMER.,P35SU0,PACU,HOME,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
64,10001884,21268656,2125-10-18 16:50:00,2125-10-20 15:15:00,NaN,EW EMER.,P20301,EMERGENCY ROOM,HOME,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2125-10-18 15:01:00,2125-10-18 18:30:00,0
65,10001884,21577720,2125-12-26 17:06:00,2125-12-27 16:52:00,NaN,EU OBSERVATION,P20301,EMERGENCY ROOM,NaN,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2125-12-26 13:50:00,2125-12-26 18:17:00,0
67,10001884,23594368,2125-11-30 16:02:00,2125-12-03 14:44:00,NaN,EW EMER.,P20301,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2125-11-30 13:25:00,2125-11-30 19:57:00,0
68,10001884,24325811,2126-11-04 00:50:00,2126-11-04 12:49:00,NaN,EU OBSERVATION,P20D4W,EMERGENCY ROOM,NaN,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2126-11-03 19:15:00,2126-11-04 12:49:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431148,19998350,21130518,2131-04-26 18:43:00,2131-04-27 15:44:00,NaN,EU OBSERVATION,P03G4D,EMERGENCY ROOM,NaN,Other,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2131-04-25 20:34:00,2131-04-26 20:17:00,0
431149,19998350,27108332,2128-02-21 09:41:00,2128-02-22 17:04:00,NaN,EU OBSERVATION,P122SD,EMERGENCY ROOM,NaN,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2128-02-21 08:08:00,2128-02-21 11:54:00,0
431197,19999379,26008899,2174-11-04 07:34:00,2174-11-05 14:05:00,NaN,EU OBSERVATION,P886Y0,EMERGENCY ROOM,NaN,Medicare,ENGLISH,MARRIED,WHITE,2174-11-04 00:25:00,2174-11-04 09:04:00,0
431199,19999442,25345411,2146-12-19 15:32:00,2146-12-27 17:20:00,NaN,ELECTIVE,P58A9J,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,SINGLE,WHITE,NaN,NaN,0


In [50]:
admission.race.unique()

array(['WHITE', 'OTHER', 'BLACK/AFRICAN AMERICAN', 'UNABLE TO OBTAIN',
       'UNKNOWN', 'WHITE - RUSSIAN', 'PORTUGUESE',
       'WHITE - OTHER EUROPEAN', 'BLACK/CAPE VERDEAN', 'ASIAN',
       'ASIAN - CHINESE', 'HISPANIC/LATINO - DOMINICAN',
       'HISPANIC/LATINO - SALVADORAN', 'HISPANIC/LATINO - PUERTO RICAN',
       'HISPANIC/LATINO - GUATEMALAN', 'ASIAN - SOUTH EAST ASIAN',
       'WHITE - BRAZILIAN', 'HISPANIC OR LATINO', 'BLACK/AFRICAN',
       'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
       'BLACK/CARIBBEAN ISLAND', 'HISPANIC/LATINO - MEXICAN',
       'PATIENT DECLINED TO ANSWER', 'HISPANIC/LATINO - CUBAN',
       'AMERICAN INDIAN/ALASKA NATIVE', 'MULTIPLE RACE/ETHNICITY',
       'WHITE - EASTERN EUROPEAN', 'ASIAN - KOREAN',
       'HISPANIC/LATINO - HONDURAN', 'ASIAN - ASIAN INDIAN',
       'HISPANIC/LATINO - COLUMBIAN',
       'HISPANIC/LATINO - CENTRAL AMERICAN', 'SOUTH AMERICAN'],
      dtype=object)

In [47]:
patients.columns

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='object')

In [3]:
#add paractice information
practice = pyreadr.read_r('../RCodes/ServerData_13Oct2020/d_practice.Rdata')
practice = practice['d_practice']
features = features.merge(practice[['practice_id','postcode_district','imd_decile','Country','County','LocalAuthority','OutputAreaClassification']], on='practice_id',how='left')
features.reset_index(drop=True, inplace=True)
features.shape

/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)


(732472, 60)

In [4]:
#BASE DATA
#extract patients from England, Wales, Scotland
features = features[(features.Country == 'England') |(features.Country == 'Wales') | (features.Country == 'Scotland')]
print(features.shape)
print('train :', features[features.Country == 'England'].shape[0])
print('val :', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0])

(724438, 60)
train : 697668
val : 26770


In [5]:
# SEX drop NA
temp = features.dropna(subset=('sex'))
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp
features.reset_index(drop=True, inplace=True)

new N:  720611
diff:  3827
train : 693841
val : 26770
diff train:  3827
diff val:  0


In [6]:
# Include only 18+
temp = features[features.age >= 18]
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp
features.reset_index(drop=True, inplace=True)

new N:  583370
diff:  137241
train : 561100
val : 22270
diff train:  132741
diff val:  4500


In [7]:
print('Missing BMI: ', features[features.height.isna()].shape[0])

Missing BMI:  48186


In [8]:
# BMI include only 15-75 and code NA as 0 (not recorded)
temp = features[((features.BMI >= 15) & (features.BMI <= 75))|(features.BMI.isna())]
temp['BMI'][temp.BMI.isna()] = 0
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp
features = temp
features.reset_index(drop=True, inplace=True)

new N:  578096
diff:  5274


/tmp/ipykernel_2812/1773957297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['BMI'][temp.BMI.isna()] = 0
/tmp/ipykernel_2812/1773957297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['BMI'][temp.BMI.isna()] = 0


train : 555964
val : 22132
diff train:  5136
diff val:  138


In [9]:
# replace NA in ethnicity and ethnic_group with 'not_recorded'
features.ethnicity[features.ethnic_group.isna()] = 'not_recorded'
features.ethnic_group[features.ethnic_group.isna()] = 'not_recorded'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [10]:
#for smoking, anyone with no record, consider them non-smoker
features.smokingStatus[features.smokingStatus.isna()] = 'never'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [11]:
#for PEF, re-assign those with no data as a 'not_recorded'
features.PEFStatus[features.PEFStatus.isna()] = 'not_recorded'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [12]:
#for Eosinophil, re-assign those with no data as a 'unknown'
features.EosinophilLevel[features.EosinophilLevel.isna()] = 'unknown'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [13]:
#for Device type, re-assign those with no data as a 'unknown'
features.DeviceType[features.DeviceType.isna()] = 'unknown'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [14]:
#BTS Step (if none recorded, it means that they are at step 0
features.BTS_step[features.BTS_step.isna()] = 0
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [15]:
#do the same thing for other continuous vars
features.average_daily_dose_ICS[features.average_daily_dose_ICS.isna()] = 0
features.prescribed_daily_dose_ICS[features.prescribed_daily_dose_ICS.isna()] = 0
features.ICS_medication_possesion_ratio[features.ICS_medication_possesion_ratio.isna()] = 0
features.Spacer[features.Spacer.isna()] = 0
features.numOCS[features.numOCS.isna()] = 0
features.numPCS[features.numPCS.isna()] = 0
features.numAntibioticsEvents[features.numAntibioticsEvents.isna()] = 0
features.numAntibioticswithLRTI[features.numAntibioticswithLRTI.isna()] = 0
features.numOCSEvents[features.numOCSEvents.isna()] = 0
features.numOCSwithLRTI[features.numOCSwithLRTI.isna()] = 0
features.numAsthmaAttacks[features.numAsthmaAttacks.isna()] = 0
features.numAcuteRespEvents[features.numAcuteRespEvents.isna()] = 0
features.PriorEducation[features.PriorEducation.isna()] = 0
features.numPCSAsthma[features.numPCSAsthma.isna()] = 0
features.CharlsonScore[features.CharlsonScore.isna()] = 0
features.numHospEvents[features.numHospEvents.isna()] = 0
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [16]:
features.columns

Index(['Unnamed: 0', 'patid', 'practice_id', 'sex', 'age', 'BMI', 'weight',
       'height', 'ethnicity', 'ethnic_group', 'smokingStatus', 'CharlsonScore',
       'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
       'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
       'count_anxiety', 'count_eczema', 'count_nasalpolyps',
       'count_paracetamol', 'count_nsaids', 'count_betablocker', 'id',
       'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',
       'PEFStatus', 'EosinophilLevel', 'BTS_step', 'average_daily_dose_ICS',
       'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio',
       'DeviceType', 'Spacer', 'numOCS', 'PriorEducation', 'numPCS',
       'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI',
       'numOCSEvents', 'numOCSwithLRTI', 'numAsthmaAttacks',
       'numAcuteRespEvents', 'numHospEvents', 'postcode_district',

## Categorized cont vars

In [17]:
#BMI
def cat_bmi(x):
    if (x>=15) & (x<18.5):
        return 'underweight'
    elif (x>=18.5) & (x<25):
        return 'normal'
    elif (x>=25) & (x<30):
        return 'overweight'
    elif (x>=50):
        return 'obese'
    
features['cat_BMI'] = features.BMI.apply(lambda x: cat_bmi(x))
features.shape

(578096, 61)

In [18]:
#Age
def cat_age(x):
    if (x>=18) & (x<35):
        return '18-34'
    elif (x>=35) & (x<55):
        return '35-54'
    elif (x>=55) & (x<75):
        return '55-74'
    elif (x>=75):
        return '>=75'
    
features['cat_age'] = features.age.apply(lambda x: cat_age(x))
features.shape

(578096, 62)

In [19]:
#Smoking
def cat_smoking(x):
    if (x=='never'):
        return 'Non Smoker'
    elif (x=='former'):
        return 'Former Smoker'
    elif (x=='current'):
        return 'Active Smoker'
features['smokingStatus'] = features.smokingStatus.apply(lambda x: cat_smoking(x))
features.shape

(578096, 62)

In [20]:
#Average daily ICS
def cat_ics(x):
    if (x==0):
        return '0'
    elif (x>=1) & (x<=200):
        return '1-200'
    elif (x>200) & (x<=400):
        return '201-400'
    elif (x>400):
        return '>400'
    
features['cat_average_daily_dose_ICS'] = features.average_daily_dose_ICS.apply(lambda x: cat_ics(x))
features['cat_prescribed_daily_dose_ICS'] = features.prescribed_daily_dose_ICS.apply(lambda x: cat_ics(x))
features.shape


(578096, 64)

In [21]:
#clean ICS medication ratio
temp = features.drop(features[features.ICS_medication_possesion_ratio==np.inf].index)
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp


new N:  577962
diff:  134
train : 555836
val : 22126
diff train:  128
diff val:  6


In [22]:
#cat ICS medication ratio
def cat_ics_medication(x):
    if x==0:
        return 'No ICS Prescribed'
    elif (x>0) & (x<0.40):
        return '>0%-39.9%'
    elif (x>=0.40) & (x<.60):
        return '40%-59.9%'
    elif (x>0.60) :
        return '>=60%'
    
features['cat_ICS_medication_possesion_ratio'] = features.ICS_medication_possesion_ratio.apply(lambda x: cat_ics_medication(x))
features.shape


(577962, 65)

In [23]:
#cat other continuous vars
def cat_continuous(x):
    if x==0:
        return '0'
    elif x==1:
        return '1'
    elif x==2:
        return '2'
    elif (x>2) :
        return '>=3'
    
features['cat_numOCS'] = features.numOCS.apply(lambda x: cat_continuous(x))
features['cat_numOCSEvents'] = features.numOCSEvents.apply(lambda x: cat_continuous(x))
features['cat_numOCSwithLRTI'] = features.numOCSwithLRTI.apply(lambda x: cat_continuous(x))
features['cat_numAcuteRespEvents'] = features.numAcuteRespEvents.apply(lambda x: cat_continuous(x))
features['cat_numAntibioticsEvents'] = features.numAntibioticsEvents.apply(lambda x: cat_continuous(x))
features['cat_numAntibioticswithLRTI'] = features.numAntibioticswithLRTI.apply(lambda x: cat_continuous(x))
features['cat_numAsthmaAttacks'] = features.numAsthmaAttacks.apply(lambda x: cat_continuous(x))
features['cat_numHospEvents'] = features.numHospEvents.apply(lambda x: cat_continuous(x))
features['cat_numPCS'] = features.numPCS.apply(lambda x: cat_continuous(x))
features['cat_numPCSAsthma'] = features.numPCSAsthma.apply(lambda x: cat_continuous(x))
features.shape

(577962, 75)

In [24]:
features.columns

Index(['Unnamed: 0', 'patid', 'practice_id', 'sex', 'age', 'BMI', 'weight',
       'height', 'ethnicity', 'ethnic_group', 'smokingStatus', 'CharlsonScore',
       'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
       'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
       'count_anxiety', 'count_eczema', 'count_nasalpolyps',
       'count_paracetamol', 'count_nsaids', 'count_betablocker', 'id',
       'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',
       'PEFStatus', 'EosinophilLevel', 'BTS_step', 'average_daily_dose_ICS',
       'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio',
       'DeviceType', 'Spacer', 'numOCS', 'PriorEducation', 'numPCS',
       'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI',
       'numOCSEvents', 'numOCSwithLRTI', 'numAsthmaAttacks',
       'numAcuteRespEvents', 'numHospEvents', 'postcode_district',

In [29]:
features.to_csv('../FinalData/cleaned_features_11072023.csv', index_label=False, index=False)

# Outcome

In [25]:
outcomes = pd.read_csv('../FinalData/data_outcomes_10072023.csv')
outcomes = outcomes.iloc[:,1:]
#select only patients included in features
outcomes = outcomes[outcomes.patid.isin(features.patid.values)]
outcomes.shape

(577962, 9)

In [26]:
outcomes["outcome_combined_6months"] = outcomes.apply(lambda x: (x["outcome_3months"] | x["outcome_6months"]), axis=1)
outcomes["outcome_combined_9months"] = outcomes.apply(lambda x: (x["outcome_combined_6months"] | x["outcome_9months"]), axis=1)
outcomes["outcome_combined_12months"] = outcomes.apply(lambda x: (x["outcome_combined_9months"] | x["outcome_12months"]), axis=1)
outcomes["outcome_combined_15months"] = outcomes.apply(lambda x: (x["outcome_combined_12months"] | x["outcome_15months"]), axis=1)
outcomes["outcome_combined_18months"] = outcomes.apply(lambda x: (x["outcome_combined_15months"] | x["outcome_18months"]), axis=1)
outcomes["outcome_combined_24months"] = outcomes.apply(lambda x: (x["outcome_combined_18months"] | x["outcome_24months"]), axis=1)
outcomes.shape

(577962, 15)

In [27]:
outcomes.shape

(577962, 15)

In [28]:
outcomes.columns

Index(['patid', 'outcome_3months', 'outcome_6months', 'outcome_9months',
       'outcome_12months', 'outcome_15months', 'outcome_18months',
       'outcome_21months', 'outcome_24months', 'outcome_combined_6months',
       'outcome_combined_9months', 'outcome_combined_12months',
       'outcome_combined_15months', 'outcome_combined_18months',
       'outcome_combined_24months'],
      dtype='object')

In [30]:
outcomes.to_csv('../FinalData/cleaned_outcomes_11072023.csv', index_label=False, index=False)